In [63]:
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import aiohttp
import asyncio  
import nest_asyncio

In [54]:


def get_category(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    # Gửi yêu cầu GET tới URL
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Lỗi tải trang: {response.status_code}")
        return

    # Kiểm tra nội dung trả về
    print(response.text[:1000])  # In ra một phần của nội dung HTML

    # Phân tích HTML bằng BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    result = []
    try:
        categories = soup.select('ul > li.pointer[data-category]')
        if not categories:
            print("Không tìm thấy danh mục trên trang.")
            return
        for category in categories:
            sub_category_list = category.select('ul > li.pointer[data-category]')
            if sub_category_list:
                try:
                    # Xử lý danh mục chính
                    if category.select('a[href] > span'):
                        main_category = category.select_one('a[href] > span').text.strip()
                        link_main = category.select_one('a[href]').get('href')
                    elif category.select('span[on] span'):  # Sửa selector ở đây
                        main_category = category.select_one('span[on] span').text.strip()
                        on_attribute = category.select_one('span[on]').get('on')
                        match = re.search(r"thor:\s*'([^']+)'", on_attribute)
                        if match:
                            link_main = f"https://iprice.vn/{match.group(1).replace('|', '/')}/"
                        else:
                            link_main = None
                    else:
                        continue
                except Exception as e:
                    print(f"Error: {e}")
                    continue
                
                sub_result = []
                for sub_category in sub_category_list:
                    try:
                        # Xử lý các phụ danh mục
                        if sub_category.select('a[href]'):
                            sub_category_element = sub_category.select_one('a[href]')
                            sub_link = sub_category_element.get('href')
                            sub_result.append({
                                "subCategory": sub_category_element.select_one('span').text,
                                "sub_link": sub_link,
                            })
                        elif sub_category.select('span[on]'):
                            sub_category_element = sub_category.select_one('span[on]')
                            on_attribute = sub_category_element.get('on')
                            match = re.search(r"thor:\s*'([^']+)'", on_attribute)
                            if match:
                                sub_link = f"https://iprice.vn/{match.group(1).replace('|', '/')}/"
                            else:
                                sub_link = None
                            sub_result.append({
                                "subCategory": sub_category_element.select_one('span').text,
                                "sub_link": sub_link,
                            })
                    except Exception as e:
                        print(f"Error (subCategory): {e}")
                
                result.append({
                    'main_category': main_category,
                    'main_link': link_main,
                    'sub_categories': sub_result
                })
    except Exception as e:
        print(f"Error: {e}")

    # Kiểm tra kết quả trước khi lưu
    if not result:
        print("Dữ liệu trích xuất rỗng.")
    else:
        # Lưu kết quả vào file JSON
        with open('categories.json', 'w', encoding='utf-8') as f:
            json.dump(result, f, ensure_ascii=False, indent=4)
        print("Dữ liệu đã được lưu vào file `categories.json`.")




In [55]:
get_category('https://iprice.vn/categories/')

<!doctype html><html amp lang="vi" class=""> <head> <meta charset="utf-8"> <title>Danh mục | iprice Việt Nam</title> <meta name="description" content="Tìm kiếm sản phẩm từ các thương hiệu hàng đầu với giá tốt nhất. Khám phá các sản phẩm mới nhất với ưu đãi tiết kiệm đến 60% tại iprice Việt Nam."/> <link rel="preconnect" href="//cdn.ampproject.org" crossorigin> <link rel="dns-prefetch" href="//cdn.ampproject.org" crossorigin> <link rel="preload" as="script" href="https://cdn.ampproject.org/v0.js"> <link rel="preconnect" href="https://p-vn.ipricegroup.com" crossorigin> <link rel="dns-prefetch" href="https://p-vn.ipricegroup.com" crossorigin> <meta name="amp-script-src" content=" "> <meta property="og:type" content="article"/> <meta property="og:site_name" content="iPrice Việt Nam"/> <meta property="og:title" content="Danh mục | iprice Việt Nam"/> <meta property="og:url" content="https://iprice.vn/categories/"/> <meta property="og:description" content="Tìm kiếm sản phẩm từ các thương hiệu

In [68]:
nest_asyncio.apply()
async def fetch_url(session, url):
    async with session.get(url) as response:
        return await response.text()

async def get_product_async(link):
    url = link + "?sort=price.net_asc"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }

    async with aiohttp.ClientSession(headers=headers) as session:
        response_text = await fetch_url(session, url)
        return response_text
def get_product(link):
    loop = asyncio.get_event_loop()
    request_start_time = time.time()
    response_text = loop.run_until_complete(get_product_async(link)) 
    request_end_time = time.time()
    print(f"Thời gian gửi yêu cầu HTTP: {request_end_time - request_start_time:.2f} giây")
    parse_start_time = time.time()
    soup = BeautifulSoup(response_text, 'html.parser')
    parse_end_time = time.time()
    print(f"Thời gian phân tích HTML: {parse_end_time - parse_start_time:.2f} giây")
    result = []
    product_list = soup.select('.non-pc')
    api = soup.select_one('amp-list.k3.i-amphtml-element.i-amphtml-built.i-amphtml-layout-container.i-amphtml-layout[src]')
    loop_start_time = time.time()
    for product in product_list:
        product_name = product.select_one('h3.zE.zF.qB').get_text(strip=True) if product.select_one('h3.zE.zF.qB') else None
        product_price = product.select_one('div.z4.nW.e7.gC').get_text(strip=True) if product.select_one('div.z4.nW.e7.gC') else None
        product_price_number = int(re.sub(r'[^\d]', '', product_price)) if product_price else None
        product_store = product.select_one('h5.zM.lc.qB.g').get_text(strip=True) if product.select_one('h5.zM.lc.qB.g') else None
        product_image = product.select_one('img.z9.z7')['src'] if product.select_one('img.z9.z7') else None
        product_rating = product.select_one('div.bF.rl.g > span.g.b.t.e5.l').get_text(strip=True) if product.select_one('div.bF.rl.g > span.g.b.t.e5.l') else None
        product_rating_number = float(product_rating) if product_rating else None
        result.append({
            'product_name': product_name,
            'product_price': product_price_number,
            'product_store': product_store,
            'product_image': product_image,
            'product_rating': product_rating_number,
            'api': api
        })
    loop_end_time = time.time()
    print(f"Thời gian duyệt qua các sản phẩm: {loop_end_time - loop_start_time:.2f} giây")
    count_dict = len(result)
    file_save_start_time = time.time()
    with open('categories_v1.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=4)
    file_save_end_time = time.time()
    print(f"Thời gian lưu dữ liệu vào file JSON: {file_save_end_time - file_save_start_time:.2f} giây")
    print("Dữ liệu đã được lưu vào file `categories_v1.json` và có " + str(count_dict))



In [ ]:
get_product("https://iprice.vn/bep-an/dao/")

In [81]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
def get_api(link):
    # Cấu hình ChromeDriver
    service = Service("chromedriver.exe")  # Đường dẫn đầy đủ đến ChromeDriver
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    # Mở trang
    driver.get(link)

    # Chờ cho đến khi thẻ <amp-list> xuất hiện
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "amp-list[src]"))
        )
        # Lấy nội dung HTML sau khi render
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        res = []
        # Tìm thẻ <amp-list>
        amp_list = soup.select_one('amp-list[src]')
        if amp_list:
            src = amp_list.get('src')
            api = 'https://iprice.vn' + src
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
                "Accept-Language": "en-US,en;q=0.9",
            }
            try:
                response = requests.get(api, headers=headers)
                if response.status_code == 200:
                    api_data = response.json()  # Lấy dữ liệu JSON

                    # Xử lý URL trong dữ liệu
                    for item in api_data.get("items", []):
                        # Xử lý trong brand
                        for brand in item.get("brand", []):
                            if "url" in brand:
                                brand["url"] = brand["url"].replace("||", "//").replace("|", "/")

                        # Xử lý trong store
                        for store in item.get("store", []):
                            if "url" in store:
                                store["url"] = store["url"].replace("||", "//").replace("|", "/")

                        # Xử lý trong color
                        for color in item.get("color", []):
                            if "url" in color:
                                color["url"] = color["url"].replace("||", "//").replace("|", "/")

                    res.append({
                        'api': api,
                        'data': api_data
                    })
                else:
                    print(f"Lỗi khi gọi API: {response.status_code}")
                    res.append({
                        'api': api,
                        'error': f"HTTP {response.status_code}"
                    })
            except Exception as e:
                print(f"Lỗi khi gọi API: {e}")
                res.append({
                    'api': api,
                    'error': str(e)
                })
        else:
            print("Không tìm thấy thẻ <amp-list>.")
        with open('api_data.json', 'w', encoding='utf-8') as f:
            json.dump(res, f, ensure_ascii=False, indent=4)
    finally:
        driver.quit()


In [82]:
get_api('https://iprice.vn/bep-an/binh-pha-ca-phe/')